In [ ]:
def discount_value(amount, t, r):
    """
    Discounts a value (or interval) 'amount' at time 't' using rate 'r'.
    'amount' can be either:
      - a float (discrete value)
      - a tuple (min_val, max_val) representing an interval
    """
    factor = (1 + r) ** t
    if isinstance(amount, tuple):  # interval (min_val, max_val)
        return (amount[0] / factor, amount[1] / factor)
    else:
        # discrete value
        return amount / factor

def add_values(v1, v2):
    """
    Adds two values that can be discrete or intervals.
    v1 and v2 can be:
       - float (discrete)
       - (float, float) (interval)
    Returns a discrete value or interval as a result.
    """
    if isinstance(v1, tuple) and isinstance(v2, tuple):
        # Interval + Interval: [a,b] + [c,d] = [a+c, b+d]
        return (v1[0] + v2[0], v1[1] + v2[1])
    elif isinstance(v1, tuple) and not isinstance(v2, tuple):
        # Interval + Discrete
        return (v1[0] + v2, v1[1] + v2)
    elif not isinstance(v1, tuple) and isinstance(v2, tuple):
        # Discrete + Interval
        return (v1 + v2[0], v1 + v2[1])
    else:
        # Both discrete
        return v1 + v2

def format_hesitant_set(h_set):
    """
    Formats a hesitant set that may contain discrete values and/or intervals.
    h_set is a list of elements, each could be a float (discrete) or a tuple (interval).
    Returns a string representing the union of all elements.
    """
    def sort_key(x):
        if isinstance(x, tuple):
            return x[0]
        else:
            return x

    # Remove duplicates
    unique_values = []
    for val in h_set:
        if val not in unique_values:
            unique_values.append(val)

    unique_values.sort(key=sort_key)

    # Create a representation similar to the one in the example
    parts = []
    for val in unique_values:
        if isinstance(val, tuple):
            # Interval
            parts.append(f"[{val[0]:.2f}, {val[1]:.2f}]")
        else:
            # Discrete
            parts.append(f"{{{val:.2f}}}")

    return " ∪ ".join(parts)

def compute_pv(hesitant_cash_flows, r):
    """
    Computes the Present Value (PV) of a Hesitant Cash Flow (HCF).
    'hesitant_cash_flows' is a list: [ (D_i, t_i), ...]
    where D_i is a list of components. Each component can be:
      - a float (discrete value)
      - an interval represented as ('interval', min_val, max_val)

    'r' is the discount rate.

    Returns a list of values (discrete or intervals) that represent
    the union of all possible PVs.
    """
    # Compute PV of each D_i separately
    pv_sets = []
    for (D, t) in hesitant_cash_flows:
        pv_D = []
        for elem in D:
            if isinstance(elem, tuple) and elem[0] == 'interval':
                # elem = ('interval', a, b)
                discounted = discount_value((elem[1], elem[2]), t, r)
                pv_D.append(discounted)
            else:
                # discrete value
                discounted = discount_value(elem, t, r)
                pv_D.append(discounted)
        pv_sets.append(pv_D)

    # Use Cartesian product to combine all PV sets
    from itertools import product

    all_combinations = product(*pv_sets)

    total_pv = []
    for combo in all_combinations:
        # combo is a tuple with one element from each pv_D
        # sum them sequentially
        current_sum = combo[0]
        for nxt in combo[1:]:
            current_sum = add_values(current_sum, nxt)
        total_pv.append(current_sum)

    return total_pv

# Example from the statement
if __name__ == "__main__":
    # Given example:
    # D_1 = {1000} U [1100,1200], t_1 = 1
    # D_2 = [950,1050] U {1250},  t_2 = 2
    # r = 0.10

    D1 = [1000, ('interval', 1100, 1200)]
    t1 = 1
    D2 = [('interval', 950, 1050), 1250]
    t2 = 2
    r = 0.10

    hcf = [ (D1, t1), (D2, t2) ]
    result = compute_pv(hcf, r)

    print("PV(F_1) =", format_hesitant_set(result))


PV(F_1) = [1694.21, 1776.86] ∪ [1785.12, 1958.68] ∪ {1942.15} ∪ [2033.06, 2123.97]
